In [2]:
import pandas as pd
import glob
import numpy as np
import sys
sys.path.append("..")

## Exploracion errores en los archivos
Exploraremos los datos y sus headers en bruto, verificando que:
* las columnas esten bien nombradas y en el orden esperado
* no existan datos vacios entre lineas
* mediciones inconsistentes o fisicamente imposibles como:
    * velocidades de viento, direcciones de viento, radiacion o precipitacion negativas
    * presion atmosferica de 0 o inferior
* el archivo leido no contenga solo 1 fila (es un dato erroneo)
* el archivo leido tenga el formato correcto (que se pueda leer)

In [5]:
headers = ['Codigo', 'Fecha', 'Hora', 'Temperatura', 'Humedad',
           'Direccion de Viento', 'Velocidad de Viento', 'Precipitacion',
           'Radiacion Solar', 'Presion Atmosferica']

def bad_headers(df):
    return any([name not in headers for name in df.columns]) or len(headers) != len(df.columns)

def has_nan(df):
    return df.isnull().any().any()

def has_bad_data(df, file):
    if not df[df['Velocidad de Viento'] < 0].empty:
        print("INCONSISTENCY.NegVel.", file)
    elif not df[df["Direccion de Viento"] < 0].empty:
        print("INCONSISTENCY.Neg.Dir.", file)
    elif not df[df["Precipitacion"] < 0].empty:
        print("INCONSISTENCY.Neg.Precip.", file)
    elif not df[df["Radiacion Solar"] < 0].empty:
        print("INCONSISTENCY.Neg.Rad.", file)
    elif not df[df["Humedad"] < 0].empty:
        print("INCONSISTENCY.Neg.Hum.", file)
    elif not df[df["Presion Atmosferica"] <= 0].empty:
        print("INCONCISTENCY.ZeroOrNeg.PAtm.", file)
    else:
        return False
    return True
    
def find_bad_excels(region, print_all=False):
    counter_bads = 0
    counter = 0
    bad_files = []
    print("===> ", region.split("/")[-1])
    for folder in glob.glob(region + "/*"):
        folder = folder.replace("\\", "/")
        for file in glob.glob(folder + "/*.xls"):
            file = file.replace("\\", "/")
            counter += 1
            try:
                df = pd.read_excel(file)
                if bad_headers(df):
                    print("ERROR.badHeader.", file)
                    print(df.count())
                    counter_bads += 1
                    bad_files.append(file)
                elif has_nan(df):
                    print("ERROR.NaN.", file)
                    print(df.count())
                    counter_bads += 1
                    bad_files.append(file)
                elif len(df) == 1:
                    print("ERROR.NoResultFile.", file)
                    print(df.count())
                    counter_bads += 1
                    bad_files.append(file)
                elif has_bad_data(df, file):
                    print(df.count())
                    counter_bads += 1
                    bad_files.append(file)
                else:
                    if print_all:
                        print("GOOD.{}".format(file))
            except:
                counter_bads += 1
                print("ERROR.file: {}".format(file))
                bad_files.append(file)   
                
    return counter, counter_bads, bad_files

In [7]:
counter = 0
counter_bads = 0
bad_files = []

raw_data_path = "../data/PAULA"


for region in glob.glob(raw_data_path + "/*"):
    region = region.replace("\\", "/")
    c, c_b, bad_files_reg = find_bad_excels(region)
    counter += c
    counter_bads += c_b
    bad_files.extend(bad_files_reg)
print("Total de archivos:", counter)
print("Total de archivos con problemas:", counter_bads, len(bad_files), len(np.unique(bad_files)))

===>  ARICA Y PARINACOTA
INCONCISTENCY.ZeroOrNeg.PAtm. ../data/PAULA/ARICA Y PARINACOTA/COLONIA JFLLUTA BAJO/Coloniafjllutabajo-01-01-2010-30-06-2010.xls
Codigo                 10715
Fecha                  10715
Hora                   10715
Temperatura            10715
Humedad                10715
Direccion de Viento    10715
Velocidad de Viento    10715
Precipitacion          10715
Radiacion Solar        10715
Presion Atmosferica    10715
dtype: int64
INCONCISTENCY.ZeroOrNeg.PAtm. ../data/PAULA/ARICA Y PARINACOTA/SOCOROMA/Socoroma 01-01-2010-30-06-2010.xls
Codigo                 8733
Fecha                  8733
Hora                   8733
Temperatura            8733
Humedad                8733
Direccion de Viento    8733
Velocidad de Viento    8733
Precipitacion          8733
Radiacion Solar        8733
Presion Atmosferica    8733
dtype: int64
ERROR.file: ../data/PAULA/ARICA Y PARINACOTA/SOCOROMA/Socoroma. 01-01-09-31-06-2009.xls
ERROR.file: ../data/PAULA/ARICA Y PARINACOTA/SOCOROMA/S

INCONSISTENCY.NegVel. ../data/PAULA/DE LA ARAUCANIA/RENAICO/Renaico-01-01-2010-30-06-2010.xls
Codigo                 15742
Fecha                  15742
Hora                   15742
Temperatura            15742
Humedad                15742
Direccion de Viento    15742
Velocidad de Viento    15742
Precipitacion          15742
Radiacion Solar        15742
Presion Atmosferica    15742
dtype: int64
INCONCISTENCY.ZeroOrNeg.PAtm. ../data/PAULA/DE LA ARAUCANIA/TOLTEN/Tolten-01-01-2010-30-06-2010.xls
Codigo                 1531
Fecha                  1531
Hora                   1531
Temperatura            1531
Humedad                1531
Direccion de Viento    1531
Velocidad de Viento    1531
Precipitacion          1531
Radiacion Solar        1531
Presion Atmosferica    1531
dtype: int64
INCONCISTENCY.ZeroOrNeg.PAtm. ../data/PAULA/DE LA ARAUCANIA/TRANAPUENTE/Tranapuente-01-01-2010-30-06-2010.xls
Codigo                 1563
Fecha                  1563
Hora                   1563
Temperatura     

ERROR.NaN. ../data/PAULA/DEL LIBERTADOR BERNARDO OHIGGINS/CODEGUA NORTE/Codeguanorte-01-07-2009-30-06-2009.xls
Codigo                 17393
Fecha                  17393
Hora                   17393
Temperatura            17393
Humedad                17393
Direccion de Viento    16601
Velocidad de Viento    17393
Precipitacion          17393
Radiacion Solar        17393
Presion Atmosferica    17393
dtype: int64
ERROR.file: ../data/PAULA/DEL LIBERTADOR BERNARDO OHIGGINS/CONTAUCO/Contauco-01-01-2009-30-06-2009.xls
INCONCISTENCY.ZeroOrNeg.PAtm. ../data/PAULA/DEL LIBERTADOR BERNARDO OHIGGINS/EL PANGUI/Elpangui-01-01-2009-30-06-2009.xls
Codigo                 17372
Fecha                  17372
Hora                   17372
Temperatura            17372
Humedad                17372
Direccion de Viento    17372
Velocidad de Viento    17372
Precipitacion          17372
Radiacion Solar        17372
Presion Atmosferica    17372
dtype: int64
INCONCISTENCY.ZeroOrNeg.PAtm. ../data/PAULA/DEL LIBERTADOR

ERROR.NaN. ../data/PAULA/DEL LIBERTADOR BERNARDO OHIGGINS/REQUINOA/REQUINOA-01-01-2009-30-06-2009.xls
Codigo                 13689
Fecha                  13689
Hora                   13689
Temperatura            13689
Humedad                13689
Direccion de Viento    13114
Velocidad de Viento    13689
Precipitacion          13689
Radiacion Solar        13689
Presion Atmosferica    13689
dtype: int64
ERROR.NaN. ../data/PAULA/DEL LIBERTADOR BERNARDO OHIGGINS/REQUINOA/REQUINOA-01-07-2009-31-12-2009.xls
Codigo                 13689
Fecha                  13689
Hora                   13689
Temperatura            13689
Humedad                13689
Direccion de Viento    13114
Velocidad de Viento    13689
Precipitacion          13689
Radiacion Solar        13689
Presion Atmosferica    13689
dtype: int64
INCONCISTENCY.ZeroOrNeg.PAtm. ../data/PAULA/DEL LIBERTADOR BERNARDO OHIGGINS/SAN FERNANDO/Sanfernando-01-01-2009-30-06-2009-xls.xls
Codigo                 15588
Fecha                  15588


INCONCISTENCY.ZeroOrNeg.PAtm. ../data/PAULA/DEL MAULE/LA ESPERANZA/Laesperanza-01-01-2010-30-06-2010.xls
Codigo                 15980
Fecha                  15980
Hora                   15980
Temperatura            15980
Humedad                15980
Direccion de Viento    15980
Velocidad de Viento    15980
Precipitacion          15980
Radiacion Solar        15980
Presion Atmosferica    15980
dtype: int64
INCONCISTENCY.ZeroOrNeg.PAtm. ../data/PAULA/DEL MAULE/LA ESPERANZA/Laesperanza-01-07-2009-31-12-2009.xls
Codigo                 17450
Fecha                  17450
Hora                   17450
Temperatura            17450
Humedad                17450
Direccion de Viento    17450
Velocidad de Viento    17450
Precipitacion          17450
Radiacion Solar        17450
Presion Atmosferica    17450
dtype: int64
INCONCISTENCY.ZeroOrNeg.PAtm. ../data/PAULA/DEL MAULE/LA HUERTA/Lahuerta-01-01-2009-30-06-2009.xls
Codigo                 17372
Fecha                  17372
Hora                   1737

ERROR.badHeader. ../data/PAULA/DEL MAULE/PENCAHUE/Pencahue-01-01-2010-30-06-2010.xls
Codigo                 16197
Fecha                  16197
Hora                   16197
Temperatura            16197
Humedad                16197
Direccion de Viento    16197
Velocidad de Viento    16197
Precipitacion          16197
Radiacion Solar        16197
Presion Atmosferica    16197
Unnamed: 10              192
Unnamed: 11                2
Unnamed: 12                2
dtype: int64
INCONCISTENCY.ZeroOrNeg.PAtm. ../data/PAULA/DEL MAULE/RETIRO/Retiro-01-01-2009-30-06-2009.xls
Codigo                 17372
Fecha                  17372
Hora                   17372
Temperatura            17372
Humedad                17372
Direccion de Viento    17372
Velocidad de Viento    17372
Precipitacion          17372
Radiacion Solar        17372
Presion Atmosferica    17372
dtype: int64
INCONCISTENCY.ZeroOrNeg.PAtm. ../data/PAULA/DEL MAULE/RETIRO/Retiro-01-01-2010-30-06-2010.xls
Codigo                 16622
Fecha

ERROR.NaN. ../data/PAULA/METROPOLITANA/CURACAVI/Curacavi-01-01-2009-30-06-2009.xls
Codigo                 13078
Fecha                  13078
Hora                   13078
Temperatura            13078
Humedad                13078
Direccion de Viento    12927
Velocidad de Viento    13078
Precipitacion          13078
Radiacion Solar        13078
Presion Atmosferica    13078
dtype: int64
INCONCISTENCY.ZeroOrNeg.PAtm. ../data/PAULA/METROPOLITANA/EL CARMELO/Elcarmelo-01-01-2010-30-06-2010.xls
Codigo                 17278
Fecha                  17278
Hora                   17278
Temperatura            17278
Humedad                17278
Direccion de Viento    17278
Velocidad de Viento    17278
Precipitacion          17278
Radiacion Solar        17278
Presion Atmosferica    17278
dtype: int64
INCONCISTENCY.ZeroOrNeg.PAtm. ../data/PAULA/METROPOLITANA/EL CARMELO/Elcarmelo-01-07-2009-31-12-2009.xls
Codigo                 6516
Fecha                  6516
Hora                   6516
Temperatura      

### Posibles correcciones
* Para las tablas con malos headers basta con renombrarlos manualmente
* Para las tablas con NaN, tenemos 2 opciones:
    * OPCION 1: borrar la fila especifica que contiene el dato NaN, esto puede causar series de tiempo irregulares
    * OPCION 2: interpolar el dato perdido (lo mas facil seria una interponacion lineal entre el dato anterior y el siguiente, si es el ultimo borrarlo)

##### por ahora es recomendable evitar estos excels para que realicen sus exploraciones

## Exploracion inconsistencias en los codigos de estaciones

Los datos originales traen las estaciones por carpetas, dentro de la cual se pueden encontrar varios archivos que comprenden distintos periodos de tiempo. Estos archivos deben todos corresponder a la misma estacion y por tanto todos deben indicar un mismo codigo. Aqui exploraremos si existen estaciones/carpetas que contengar datos/archivos relacionados a mas de 1 codigo, lo que significaria una inconsistencia


In [15]:
headers = ['Codigo', 'Fecha', 'Hora', 'Temperatura', 'Humedad',
           'Direccion de Viento', 'Velocidad de Viento', 'Precipitacion',
           'Radiacion Solar', 'Presion Atmosferica']

def has_double_code(estacion):
    print("--> ", estacion.split("/")[-1])
    codigo = None
    res = False
    for file in glob.glob(estacion + "/*.xls"):
        file = file.replace("\\", "/")
        if file not in bad_files:
            df = pd.read_excel(file)
            df_code = np.unique(list(df['Codigo']))
            if len(df_code) == 1:
                if codigo is None:
                    codigo = df_code[0]
                else:
                    if codigo != df_code[0]:
                        res = True
            else:
                res = True
    return res

def check_for_not_unique_codes(region):
    bfs = []
    print(region.split("/")[-1])
    for folder in glob.glob(region + "/*"):
        folder = folder.replace("\\", "/")
        if has_double_code(folder):
            files = glob.glob(folder + "/*.xls")
            print("DOUBLE.CODE.STATION.", folder.split("/")[-1])
            bfs.extend(files)
    return bfs

In [16]:
bad_files_double_code = []
for region in glob.glob(raw_data_path + "/*"):
    region = region.replace("\\", "/")
    bad_files_reg = check_for_not_unique_codes(region)
    bad_files_double_code.extend(bad_files_reg)

ARICA Y PARINACOTA
-->  AZAPA1
-->  AZAPA2
-->  COLONIA JFLLUTA BAJO
-->  PUROCHILE LLUTA MEDIO
-->  SOCOROMA
-->  TRUFA AZAPA MEDIO
ATACAMA
-->  ALTO DEL CARMEN
-->  ATACAMA
DOUBLE.CODE.STATION. ATACAMA
-->  Atacama.xls
-->  BODEGA
-->  COPIAPO
-->  HORNITOS
-->  JOTABECHE
-->  LAS JUNTAS
-->  LOS LOROS
-->  Region-Atacama.xls
-->  Region-Atacama1.xls
-->  TRANQUE-LAUTARO
-->  VALLENAR
AYSEN DEL GENERAL DEL C.I.CAMPO
-->  AYSEN
-->  CHILECHICO
-->  COYHAIQUE
COQUIMBO
-->  ALGARROBAL
-->  CARACHILLA
-->  COMBARBALA
-->  EL PALQUI
-->  ILLAPEL
-->  MONTE PATRIA
-->  PAIHUANO
-->  PALQUI
-->  PEDREGAL
-->  PUNITAQUI
-->  RECOLETA
-->  SALAMANCA
-->  SAN JULIAN
-->  VICUÑA
DE LA ARAUCANIA
-->  CARAHUE
-->  COLLIPULLI
-->  CUNCO
-->  CURACAUTIN
-->  FREIRE
-->  GALVARINO
-->  GORBEA
-->  GORBEA2
-->  LOMCOCHE
-->  NUEVA IMPERIAL
-->  POCOYAN
-->  PUCON
-->  PUERTO SAAVEDRA
-->  RENAICO
-->  TOLTEN
-->  TRAIGEN
-->  TRANAPUENTE
DE LOS LAGOS
-->  BUTALCURA
-->  FRUTILLAR
-->  OSORNO
-->  PUR

In [19]:
final_bad_files = bad_files
final_bad_files.extend(bad_files_double_code)
data_folder = "../data"
np.save(data_folder + "/bad_files.npy", final_bad_files)

### Resultados

* La estacion 'ATACAMA" presenta mas de un codigo, esto significa que algun archivo se traspapelo o que existe una inconsistencia en los datos recibidos. Para corregir esta estacion es necesario realizar una reunion con el cliente
* el path a los archivos con problemas (iniciando desde ./PAULA) se guardara en un .npy para el futuro